In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

Load a CSV file

In [2]:
def load_csv(file_path):
  
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {file_path}")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

initial exploration of the data

In [3]:
def explore_data(df, filename):
  
    print(f"\n--- Exploring {filename} ---")
    print(f"Shape: {df.shape}")
    print("\nData Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nBasic Statistics:")
    print(df.describe().T)
    print("\nSample Data:")
    print(df.head())

Handle missing values

In [4]:
def handle_missing_values(df, numeric_strategy='mean', categorical_strategy='most_frequent'):
  
    df_copy = df.copy()
    
    # Separate numerical and categorical columns
    numerical_cols = df_copy.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df_copy.select_dtypes(include=['object', 'category']).columns
    
    # Handle numerical missing values
    if len(numerical_cols) > 0:
        num_imputer = SimpleImputer(strategy=numeric_strategy)
        df_copy[numerical_cols] = num_imputer.fit_transform(df_copy[numerical_cols])
        
    # Handle categorical missing values
    if len(categorical_cols) > 0:
        cat_imputer = SimpleImputer(strategy=categorical_strategy)
        df_copy[categorical_cols] = cat_imputer.fit_transform(df_copy[categorical_cols])
    
    return df_copy

  Encode categorical variables

In [5]:
def encode_categorical(df, cols_to_encode=None, method='onehot'):
 
    df_copy = df.copy()
    
    if cols_to_encode is None:
        cols_to_encode = df_copy.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if method == 'onehot':
        for col in cols_to_encode:
            if col in df_copy.columns:
                dummies = pd.get_dummies(df_copy[col], prefix=col, drop_first=True)
                df_copy = pd.concat([df_copy, dummies], axis=1)
                df_copy.drop(col, axis=1, inplace=True)
    
    elif method == 'label':
        for col in cols_to_encode:
            if col in df_copy.columns:
                df_copy[col] = df_copy[col].astype('category').cat.codes
    
    return df_copy


Scale numerical features

In [6]:
def scale_features(df, cols_to_scale=None, method='standard'):
  
    df_copy = df.copy()
    
    if cols_to_scale is None:
        cols_to_scale = df_copy.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'minmax':
        scaler = MinMaxScaler()
    
    df_copy[cols_to_scale] = scaler.fit_transform(df_copy[cols_to_scale])
    
    return df_copy

 Detect and handle outliers in numerical columns

In [7]:
def handle_outliers(df, cols=None, method='iqr', threshold=1.5):
   
    df_copy = df.copy()
    
    if cols is None:
        cols = df_copy.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    for col in cols:
        if method == 'iqr':
            Q1 = df_copy[col].quantile(0.25)
            Q3 = df_copy[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            # Capping outliers at bounds
            df_copy[col] = np.where(df_copy[col] > upper_bound, upper_bound, df_copy[col])
            df_copy[col] = np.where(df_copy[col] < lower_bound, lower_bound, df_copy[col])
            
        elif method == 'zscore':
            mean = df_copy[col].mean()
            std = df_copy[col].std()
            z_scores = (df_copy[col] - mean) / std
            df_copy[col] = np.where(np.abs(z_scores) > threshold, mean, df_copy[col])
    
    return df_copy

Create new features from existing ones

In [8]:
def create_features(df, date_cols=None):
   
    df_copy = df.copy()
    
    if date_cols:
        for col in date_cols:
            if col in df_copy.columns:
                try:
                    df_copy[col] = pd.to_datetime(df_copy[col])
                    df_copy[f'{col}_year'] = df_copy[col].dt.year
                    df_copy[f'{col}_month'] = df_copy[col].dt.month
                    df_copy[f'{col}_day'] = df_copy[col].dt.day
                    df_copy[f'{col}_dayofweek'] = df_copy[col].dt.dayofweek
                except Exception as e:
                    print(f"Error processing date column {col}: {e}")
    
    return df_copy

Drop unnecessary columns

In [10]:
def drop_columns(df, cols_to_drop):
    return df.drop(columns=cols_to_drop, errors='ignore')

preprocessing pipeline for a single CSV file

In [11]:
def preprocess_file(file_path, date_cols=None, cols_to_drop=None, categorical_encoding='onehot', 
                   scaling_method='standard', handle_missing=True, handle_outliers_flag=True):
   
    # Load data
    df = load_csv(file_path)
    if df is None:
        return None
    
    # Explore the data
    explore_data(df, file_path)
    
    # Handle missing values if specified
    if handle_missing:
        df = handle_missing_values(df)
        print(f"Missing values handled in {file_path}")
    
    # Create features from date columns if specified
    if date_cols:
        df = create_features(df, date_cols)
        print(f"Features created from date columns in {file_path}")
    
    # Drop unnecessary columns if specified
    if cols_to_drop:
        df = drop_columns(df, cols_to_drop)
        print(f"Dropped columns: {cols_to_drop} from {file_path}")
    
    # Handle outliers if specified
    if handle_outliers_flag:
        df = handle_outliers(df)
        print(f"Outliers handled in {file_path}")
    
    # Encode categorical variables
    df = encode_categorical(df, method=categorical_encoding)
    print(f"Categorical variables encoded in {file_path}")
    
    # Scale numerical features
    df = scale_features(df, method=scaling_method)
    print(f"Numerical features scaled in {file_path}")
    
    print(f"Preprocessing complete for {file_path}")
    return df

Set file paths

In [12]:
file_path1 = "dataset/computed_insight_success_of_active_sellers.csv"
file_path2 = "dataset/summer-products-with-rating-and-performance_2020-08.csv"
file_path3 = "dataset/unique-categories.csv"
file_path4 = "dataset/unique-categories.sorted-by-count.csv"

File 1

In [15]:
print("\n===== Processing File 1: computed_insight_success_of_active_sellers.csv =====")
df1 = load_csv(file_path1)
if df1 is not None:
    # Explore raw data
    explore_data(df1, file_path1)
    
    # Handle missing values
    df1_clean = handle_missing_values(df1)
    print("Missing values handled")
    
    # Handle outliers in numerical columns
    df1_clean = handle_outliers(df1_clean)
    print("Outliers handled")
    
    # Encode categorical variables if any
    df1_clean = encode_categorical(df1_clean)
    print("Categorical variables encoded")
    
    # Scale numerical features
    df1_processed = scale_features(df1_clean)
    print("Numerical features scaled")
    
    print("Preprocessing complete for File 1")
    print("Sample of processed data:")
    print(df1_processed.head())
    
    # Save processed file 
    df1_processed.to_csv("preprocessed_dataset/processed_computed_insight_success_of_active_sellers.csv", index=False)


===== Processing File 1: computed_insight_success_of_active_sellers.csv =====
Successfully loaded dataset/computed_insight_success_of_active_sellers.csv

--- Exploring dataset/computed_insight_success_of_active_sellers.csv ---
Shape: (958, 13)

Data Types:
merchantid                  object
listedproducts               int64
totalunitssold               int64
meanunitssoldperproduct    float64
rating                     float64
merchantratingscount       float64
meanproductprices          float64
meanretailprices           float64
averagediscount            float64
meandiscount               float64
meanproductratingscount    float64
totalurgencycount          float64
urgencytextrate            float64
dtype: object

Missing Values:
merchantid                   0
listedproducts               0
totalunitssold               0
meanunitssoldperproduct      0
rating                       0
merchantratingscount         0
meanproductprices            0
meanretailprices             0
averaged

File 2

In [16]:
print("\n===== Processing File 2: summer-products-with-rating-and-performance_2020-08.csv =====")
df2 = load_csv(file_path2)
if df2 is not None:
    # Explore raw data
    explore_data(df2, file_path2)
    
    # Extract date features if any date columns exist
    # Assuming there might be date columns related to summer 2020
    date_cols = []
    for col in df2.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            date_cols.append(col)
    
    if date_cols:
        df2_features = create_features(df2, date_cols)
        print(f"Features created from date columns: {date_cols}")
    else:
        df2_features = df2.copy()
    
    # Handle missing values
    df2_features = handle_missing_values(df2_features)
    print("Missing values handled")
    
    # Handle outliers
    df2_features = handle_outliers(df2_features)
    print("Outliers handled")
    
    # Encode categorical variables
    df2_features = encode_categorical(df2_features)
    print("Categorical variables encoded")
    
    # Scale numerical features - using MinMax since ratings often have bounded ranges
    df2_processed = scale_features(df2_features, method='minmax')
    print("Numerical features scaled using MinMax scaling")
    
    print("Preprocessing complete for File 2")
    print("Sample of processed data:")
    print(df2_processed.head())
    
    # Save processed file if needed
    df2_processed.to_csv("preprocessed_dataset/processed_summer_products.csv", index=False)


===== Processing File 2: summer-products-with-rating-and-performance_2020-08.csv =====
Successfully loaded dataset/summer-products-with-rating-and-performance_2020-08.csv

--- Exploring dataset/summer-products-with-rating-and-performance_2020-08.csv ---
Shape: (1573, 43)

Data Types:
title                            object
title_orig                       object
price                           float64
retail_price                      int64
currency_buyer                   object
units_sold                        int64
uses_ad_boosts                    int64
rating                          float64
rating_count                      int64
rating_five_count               float64
rating_four_count               float64
rating_three_count              float64
rating_two_count                float64
rating_one_count                float64
badges_count                      int64
badge_local_product               int64
badge_product_quality             int64
badge_fast_shipping               

File 3

In [18]:
print("\n===== Processing File 3: unique-categories.csv =====")
df3 = load_csv(file_path3)
if df3 is not None:
    # Explore raw data
    explore_data(df3, file_path3)
    
    # Handle missing values
    df3_clean = handle_missing_values(df3)
    print("Missing values handled")
    
    # For category data, we might want to use label encoding instead of one-hot
    df3_processed = encode_categorical(df3_clean, method='label')
    print("Categorical variables encoded using label encoding")
    
    print("Preprocessing complete for File 3")
    print("Sample of processed data:")
    print(df3_processed.head())
    
    # Save processed file if needed
    df3_processed.to_csv("preprocessed_dataset/processed_unique_categories.csv", index=False)


===== Processing File 3: unique-categories.csv =====
Successfully loaded dataset/unique-categories.csv

--- Exploring dataset/unique-categories.csv ---
Shape: (2620, 1)

Data Types:
tag    object
dtype: object

Missing Values:
tag    0
dtype: int64

Basic Statistics:
    count unique               top freq
tag  2620   2620  #fashion #shorts    1

Sample Data:
                tag
0  #fashion #shorts
1  #fashion #tshirt
2        #Sport Bra
3   #Summer Clothes
4        #top #crop
Missing values handled
Categorical variables encoded using label encoding
Preprocessing complete for File 3
Sample of processed data:
   tag
0    2
1    3
2    0
3    1
4    4


File 4

In [19]:
print("\n===== Processing File 4: unique-categories.sorted-by-count.csv =====")
df4 = load_csv(file_path4)
if df4 is not None:
    # Explore raw data
    explore_data(df4, file_path4)
    
    # Handle missing values
    df4_clean = handle_missing_values(df4)
    print("Missing values handled")
    
    # For count data, encode categorical and then normalize the counts
    df4_encoded = encode_categorical(df4_clean, method='label')
    print("Categorical variables encoded using label encoding")
    
    # Scale numerical features - MinMax may work well for count data
    df4_processed = scale_features(df4_encoded, method='minmax')
    print("Numerical features scaled using MinMax scaling")
    
    print("Preprocessing complete for File 4")
    print("Sample of processed data:")
    print(df4_processed.head())
    
    # Save processed file if needed
    df4_processed.to_csv("preprocessed_dataset/processed_unique_categories_counts.csv", index=False)

print("\n===== All files preprocessing complete =====")


===== Processing File 4: unique-categories.sorted-by-count.csv =====
Successfully loaded dataset/unique-categories.sorted-by-count.csv

--- Exploring dataset/unique-categories.sorted-by-count.csv ---
Shape: (2620, 2)

Data Types:
count       int64
keyword    object
dtype: object

Missing Values:
count      0
keyword    0
dtype: int64

Basic Statistics:
        count      mean        std  min  25%  50%  75%     max
count  2620.0  10.44084  60.528466  1.0  1.0  1.0  4.0  1321.0

Sample Data:
   count          keyword
0   1321           Summer
1   1315  Women's Fashion
2   1082          Fashion
3    961            Women
4    905           Casual
Missing values handled
Categorical variables encoded using label encoding
Numerical features scaled using MinMax scaling
Preprocessing complete for File 4
Sample of processed data:
      count  keyword
0  1.000000      544
1  0.995455      648
2  0.818939      204
3  0.727273      629
4  0.684848      112

===== All files preprocessing complete =

In [20]:
print("\nSummary of processed datasets:")
print(f"File 1 shape: {df1_processed.shape if 'df1_processed' in locals() else 'Not processed'}")
print(f"File 2 shape: {df2_processed.shape if 'df2_processed' in locals() else 'Not processed'}")
print(f"File 3 shape: {df3_processed.shape if 'df3_processed' in locals() else 'Not processed'}")
print(f"File 4 shape: {df4_processed.shape if 'df4_processed' in locals() else 'Not processed'}")


Summary of processed datasets:
File 1 shape: (958, 969)
File 2 shape: (1573, 11951)
File 3 shape: (2620, 1)
File 4 shape: (2620, 2)
